In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import requests
from datetime import datetime, timedelta

import time

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn import preprocessing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
loadarea = pd.read_csv('/kaggle/input/energy-data-in-taiwan-taipower/loadarea_20170101-20210522.csv')
loadarea = loadarea.pivot_table(columns='area',index='datetime',values='load')
loadarea.index= pd.to_datetime(loadarea.index)
loadarea

In [ ]:
weatherData = pd.read_csv('/kaggle/input/energy-data-in-taiwan-taipower/weatherData_20170101-20210522.csv', encoding='utf8')
weatherData

In [ ]:
temperatureData = weatherData.pivot_table(index='ObsTime',columns='station',values='Temperature')
temperatureData.index = pd.to_datetime(temperatureData.index)
temperatureData.columns = 'temperature_'+temperatureData.columns
temperatureData

In [ ]:
loadarea.loc['2020'].resample('H').mean().sum(axis=1).iplot()
loadarea.loc['2020'].resample('D').mean().sum(axis=1).iplot()

In [ ]:
df_holiday = pd.DataFrame()

for year in np.arange(2017,2022):
    url = 'https://www.timeanddate.com/holidays/taiwan/'+str(year)
    r = requests.get(url)
    df_holiday_temp = pd.read_html(r.text)[0]
    df_holiday_temp.columns = df_holiday_temp.columns.get_level_values(0)
    df_holiday_temp = df_holiday_temp.dropna(how='all')
    df_holiday_temp = df_holiday_temp[['Date', 'Name', 'Type']]
    df_holiday_temp['Date'] = str(year) + ' ' + df_holiday_temp['Date']
    df_holiday_temp['Date'] = pd.to_datetime(df_holiday_temp['Date'], errors='coerce')
    df_holiday_temp = df_holiday_temp[~df_holiday_temp['Date'].isna()]
    
    df_holiday = pd.concat([df_holiday, df_holiday_temp], axis=0, ignore_index=True)

df_holiday = df_holiday[df_holiday['Type']=='National holiday']
df_holiday = df_holiday.drop_duplicates(subset=['Date'])
df_holiday = df_holiday.set_index('Date')
df_holiday.loc['2021-12-31', ['Name','Type']]=np.nan
df_holiday = df_holiday.asfreq('D')
df_holiday.loc[df_holiday.index.weekday>=5, 'Name'] = 'weekend'
df_holiday.loc[df_holiday.index.weekday>=5, 'Type'] = 'weekend'
df_holiday.columns = 'holiday_' + df_holiday.columns

df_holiday = df_holiday.reset_index()
df_holiday = df_holiday.rename(columns={'Date':'date'}) 

df_holiday = df_holiday.fillna('regular')

df_holiday

In [ ]:
dataset_hourly = loadarea.resample('H').mean()
dataset_hourly.columns = 'load_'+dataset_hourly.columns
dataset_hourly = dataset_hourly.merge(temperatureData, left_index=True, right_index=True)

index_temp = dataset_hourly.index.copy()

dataset_hourly['date'] = pd.to_datetime(dataset_hourly.index.date)
dataset_hourly = dataset_hourly.merge(df_holiday, on='date')
dataset_hourly = dataset_hourly.drop('date',axis=1)
dataset_hourly.index = index_temp

#dataset_hourly['holiday_Name'] = preprocessing.LabelEncoder().fit_transform(dataset_hourly['holiday_Name'])
#dataset_hourly['holiday_Type'] = preprocessing.LabelEncoder().fit_transform(dataset_hourly['holiday_Type'])

dataset_hourly

In [ ]:
dataset_hourly['load_total'] = dataset_hourly['load_north']+ dataset_hourly['load_east']+ dataset_hourly['load_central']+ dataset_hourly['load_south']
dataset_hourly['temperature_avg'] = dataset_hourly[['temperature_466920_臺北', 'temperature_466990_花蓮','temperature_467410_臺南', 'temperature_467490_臺中']].mean(axis=1)

In [ ]:
# Scatter plot for demand and outdoor temperature
df_plot = dataset_hourly.copy()
df_plot = df_plot.resample('D').mean()

index_temp = df_plot.index.copy()
df_plot['date'] = pd.to_datetime(df_plot.index.date)
df_plot = df_plot.merge(df_holiday, on='date')
df_plot = df_plot.drop('date',axis=1)
df_plot.index = index_temp

df_plot = df_plot.rename(columns={'holiday_Type':'day_type'})

df_plot['weekday'] = df_plot.index.weekday
df_plot['year'] = df_plot.index.year

sns.lmplot(x="temperature_avg", y="load_total",
           data=df_plot, order=2, scatter_kws={'alpha':0.25}, ci=None)

In [ ]:
# Scatter plot for demand and outdoor temperature
df_plot = dataset_hourly.copy()
df_plot = df_plot.resample('D').mean()

index_temp = df_plot.index.copy()
df_plot['date'] = pd.to_datetime(df_plot.index.date)
df_plot = df_plot.merge(df_holiday, on='date')
df_plot = df_plot.drop('date',axis=1)
df_plot.index = index_temp

df_plot = df_plot.rename(columns={'holiday_Type':'day_type'})

df_plot['weekday'] = df_plot.index.weekday
df_plot['year'] = df_plot.index.year

sns.lmplot(x="temperature_avg", y="load_total",hue='day_type',
           data=df_plot, order=2, scatter_kws={'alpha':0.25}, ci=None)

In [ ]:
for  year in np.arange(2017,2021):

    # Scatter plot for demand and outdoor temperature
    df_plot = dataset_hourly.loc[str(year)].copy()
    #df_plot = df_plot[dataset_hourly['holiday_Type']=='regular']
    df_plot = df_plot.resample('D').mean()
    
    index_temp = df_plot.index.copy()
    df_plot['date'] = pd.to_datetime(df_plot.index.date)
    df_plot = df_plot.merge(df_holiday, on='date')
    df_plot = df_plot.drop('date',axis=1)
    df_plot.index = index_temp
    
    df_plot = df_plot.sort_values('holiday_Type')

    sns.lmplot(x="temperature_avg", y="load_total", hue="holiday_Type",
               data=df_plot, order=2, scatter_kws={'alpha':0.5}, ci=None)
    plt.show()

In [ ]:
# Scatter plot for demand and outdoor temperature
df_plot = dataset_hourly[dataset_hourly['holiday_Type']=='regular'].copy()
df_plot = df_plot.resample('D').mean()
df_plot['weekday'] = df_plot.index.weekday
df_plot['year'] = df_plot.index.year

sns.lmplot(x="temperature_avg", y="load_total", hue="year",
           data=df_plot, order=2, scatter_kws={'alpha':0.5}, ci=None)

In [ ]:
# Scatter plot for demand and outdoor temperature
df_plot = dataset_hourly[dataset_hourly['holiday_Type']=='weekend'].copy()
df_plot = df_plot.resample('D').mean()
df_plot['weekday'] = df_plot.index.weekday
df_plot['year'] = df_plot.index.year

sns.lmplot(x="temperature_avg", y="load_total", hue="year",
           data=df_plot, order=2, scatter_kws={'alpha':0.5}, ci=None)

In [ ]:
# Scatter plot for demand and outdoor temperature
df_plot = dataset_hourly[dataset_hourly['holiday_Type']=='National holiday'].copy()
df_plot = df_plot.resample('D').mean()
df_plot['weekday'] = df_plot.index.weekday
df_plot['year'] = df_plot.index.year

sns.lmplot(x="temperature_avg", y="load_total", hue="year",
           data=df_plot, order=2, scatter_kws={'alpha':0.5}, ci=None)